In [1]:
import cv2
import numpy as np
import os
import sys
import tensorflow as tf

from sklearn.model_selection import train_test_split

from tqdm import tqdm

EPOCHS = 10
IMG_WIDTH = 30
IMG_HEIGHT = 30
NUM_CATEGORIES = 3
TEST_SIZE = 0.4


def main():

    # Check command-line arguments
    #if len(sys.argv) not in [2, 3]:
    #    sys.exit("Usage: python traffic.py data_directory [model.h5]")

    # Get image arrays and labels for all image files
    #images, labels = load_data(sys.argv[1])
    sys_argv1 = "gtsrb-small"
    images, labels = load_data(sys_argv1)

    # Split data into training and testing sets
    labels = tf.keras.utils.to_categorical(labels)
    x_train, x_test, y_train, y_test = train_test_split(
        np.array(images), np.array(labels), test_size=TEST_SIZE
    )

    # Get a compiled neural network
    model = get_model()

    # Fit model on training data
    model.fit(x_train, y_train, epochs=EPOCHS)

    # Evaluate neural network performance
    model.evaluate(x_test,  y_test, verbose=2)

    # Save model to file
    #if len(sys.argv) == 3:
    #    filename = sys.argv[2]
    #    model.save(filename)
    #    print(f"Model saved to {filename}.")
    sys_argv2 = "model.sav"
    filename = sys_argv2
    model.save(filename)
    print(f"Model saved to {filename}.")


def load_data(data_dir):
    """
    Load image data from directory `data_dir`.

    Assume `data_dir` has one directory named after each category, numbered
    0 through NUM_CATEGORIES - 1. Inside each category directory will be some
    number of image files.

    Return tuple `(images, labels)`. `images` should be a list of all
    of the images in the data directory, where each image is formatted as a
    numpy ndarray with dimensions IMG_WIDTH x IMG_HEIGHT x 3. `labels` should
    be a list of integer labels, representing the categories for each of the
    corresponding `images`.
    """
    # CG: initiate resulting lists:
    images = list()
    labels = list()
    filenames = list()
    imagelist = list()
    badreads  = list()

    #CG: assume that data_dir will contain one directory named after each category, numbered 0 through NUM_CATEGORIES - 1:
    for cat in tqdm(range(NUM_CATEGORIES)):

        # CG: build search_dir by adding the category number to the starting dir:
        search_dir = data_dir + os.sep + str(cat) + os.sep

        # CG: walk down the directory tree:
        for root, dirs, files in os.walk(search_dir):

            # CG: get a list of all files in the directory:
            names = [(os.path.join(root,f)) for f in files]

            # CG: ignore empty entries:
            if len(names) == 0:
                continue

            # CG: add an entry in the arrays for each file found:
            for f in names:

                # CG: load the image file:
                img = cv2.imread (f, cv2.IMREAD_UNCHANGED)

                # CG: if failed, add to badreads list: 
                if img is None:
                    badreads.append (f)
                    continue

                # CG: add filename to list of valid files:
                filenames.append(f)

                # CG: resize the image to the desired sizes:
                img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))

                # CG: convert the image to RGB color scheme:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                # CG: add the image to the list:
                images.append(img)

                # CG: add the corresponding categorie to the list:
                labels.append(cat)

    # CG: return the lists:
    return (images, labels)


def get_model():
    """
    Returns a compiled convolutional neural network model. Assume that the
    `input_shape` of the first layer is `(IMG_WIDTH, IMG_HEIGHT, 3)`.
    The output layer should have `NUM_CATEGORIES` units, one for each category.
    """
    
    # CG: create the model as Sequential:
    model = tf.keras.models.Sequential()

    # CG: add a convolutional layer. Learn 32 filters using a 4x4 kernel, shaping input according to given image parameters:
    model.add (tf.keras.layers.Conv2D(32, (4, 4), activation="relu", input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)))

    # CG: add a convolutional layer. Learn 32 filters using a 2x2 kernel:
    model.add (tf.keras.layers.Conv2D(32, (2, 2), activation="relu"))

    # CG: max-pooling layer, using 2x2 pool size:
    model.add (tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    # CG: flatten units:
    model.add (tf.keras.layers.Flatten())

    # CG: add a hidden layer with 32X32X3 (3072) neurons and 1/3% dropout:
    model.add (tf.keras.layers.Dense(3072, activation="relu"))
    model.add (tf.keras.layers.Dropout(1/2))

    # CG: add a hidden layer with 1024 neurons:
    model.add (tf.keras.layers.Dense(1024, activation="relu"))

    # CG: add a hidden layer with 512 neurons:
    model.add (tf.keras.layers.Dense(512, activation="relu"))

    # CG: add a hidden layer with 256:
    model.add (tf.keras.layers.Dense(256, activation="relu"))

    # CG: add a hidden layer with 128:
    model.add (tf.keras.layers.Dense(128, activation="relu"))
    #model.add (tf.keras.layers.Dropout(1/3))

    # CG: add a hidden layer with 64:
    model.add (tf.keras.layers.Dense(64, activation="relu"))
    #model.add (tf.keras.layers.Dropout(1/3))

    # CG: add an output layer with output units for all NUM_CATEGORIES categories:
    model.add (tf.keras.layers.Dense(NUM_CATEGORIES, activation="softmax"))

    # CG: compile the model
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

    # CG: return the compiled model:
    return model
  

if __name__ == "__main__":
    main()


D:\Documents\Anaconda3\envs\ai50\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 16.46it/s]


Epoch 1/10
16/16 [==============================] - 2s 136ms/step - loss: 33.6659 - accuracy: 0.5298
Epoch 2/10
16/16 [==============================] - 2s 133ms/step - loss: 0.2640 - accuracy: 0.9107
Epoch 3/10
16/16 [==============================] - 2s 138ms/step - loss: 0.0429 - accuracy: 0.9861
Epoch 4/10
16/16 [==============================] - 2s 135ms/step - loss: 0.0134 - accuracy: 0.9960
Epoch 5/10
16/16 [==============================] - 2s 129ms/step - loss: 0.0090 - accuracy: 0.9980
Epoch 6/10
16/16 [==============================] - 2s 126ms/step - loss: 1.2557e-04 - accuracy: 1.0000
Epoch 7/10
16/16 [==============================] - 2s 130ms/step - loss: 3.8893e-05 - accuracy: 1.0000
Epoch 8/10
16/16 [==============================] - 2s 130ms/step - loss: 2.0225e-05 - accuracy: 1.0000
Epoch 9/10
16/16 [==============================] - 2s 133ms/step - loss: 2.0858e-05 - accuracy: 1.0000
Epoch 10/10
16/16 [==============================] - 2s 132ms/step - loss: 1.6880e-

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0
